In [1]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split

# CONFIGURATION
image_dir = "/kaggle/input/original/train/images"
label_dir = "/kaggle/input/original/train/labels"
bg_image_dir = "/kaggle/input/background/background_only"
bg_label_dir = "/kaggle/input/background/background_only_labels"
output_base = "/kaggle/working/combined_split"
num_bg_to_add = 180  # 👈 change this as needed

# Step 1: Get only labeled image base names (non-empty label .txts)
image_basenames = []
for fname in os.listdir(label_dir):
    if fname.endswith(".txt"):
        with open(os.path.join(label_dir, fname)) as f:
            if f.read().strip():
                image_basenames.append(fname.replace(".txt", ""))

#  Step 2: Split labeled data
train_val, test = train_test_split(image_basenames, test_size=0.1, random_state=42)
train, val = train_test_split(train_val, test_size=0.2222, random_state=42)  # ~20% val

# Step 3: Prepare output folders
def copy_split(split_names, split_type):
    img_out = os.path.join(output_base, split_type, "images")
    lbl_out = os.path.join(output_base, split_type, "labels")
    os.makedirs(img_out, exist_ok=True)
    os.makedirs(lbl_out, exist_ok=True)

    for name in split_names:
        shutil.copy(os.path.join(image_dir, f"{name}.jpg"), os.path.join(img_out, f"{name}.jpg"))
        shutil.copy(os.path.join(label_dir, f"{name}.txt"), os.path.join(lbl_out, f"{name}.txt"))

copy_split(train, "train")
copy_split(val, "valid")
copy_split(test, "test")

#  Step 4: Select & copy a limited number of background-only images to training set
bg_images = [f for f in os.listdir(bg_image_dir) if f.endswith(".jpg")]
random.shuffle(bg_images)
bg_images_to_add = bg_images[:num_bg_to_add]

train_img_dir = os.path.join(output_base, "train/images")
train_lbl_dir = os.path.join(output_base, "train/labels")

for img_name in bg_images_to_add:
    base = os.path.splitext(img_name)[0]
    img_path = os.path.join(bg_image_dir, img_name)
    lbl_path = os.path.join(bg_label_dir, f"{base}.txt")
    shutil.copy(img_path, os.path.join(train_img_dir, img_name))
    shutil.copy(lbl_path, os.path.join(train_lbl_dir, f"{base}.txt"))

print(f"✅ Final split: {len(train)} train (+{len(bg_images_to_add)} bg), {len(val)} val, {len(test)} test")

✅ Final split: 346 train (+180 bg), 99 val, 50 test


In [2]:
yaml_content = """
train: /kaggle/working/combined_split/train/images
val: /kaggle/working/combined_split/valid/images
test: /kaggle/working/combined_split/test/images

nc: 4
names: ['Absence_Head', 'Absence_Tail', 'Head_Rivet', 'Tail_Rivet']
"""

yaml_path = "/kaggle/working/rivet_detection.yaml"
with open(yaml_path, "w") as f:
    f.write(yaml_content.strip())

print(f"✅ YAML file written to: {yaml_path}")

✅ YAML file written to: /kaggle/working/rivet_detection.yaml


In [ ]:
!pip install ultralytics --upgrade --quiet

import os
from ultralytics import YOLO


DATA_YAML_PATH = "/kaggle/working/rivet_detection.yaml"  
model = YOLO("yolov8s.pt").to("cuda")  

# Train the model
model.train(
    data=DATA_YAML_PATH,
    device='cuda',
    epochs=100,
    batch=8,
    imgsz=896,         
    patience=20,
    optimizer='SGD',
    lr0=0.002,
    lrf=0.1,
    weight_decay=0.0005,
    cos_lr=True,
    save_period=10,
    workers=4,
    amp=True,

    mosaic=1.0,
    auto_augment='randaugment',
    augment=True,

    hsv_h=0.01,
    hsv_s=0.8,
    hsv_v=0.4,

    flipud=0.5,
    fliplr=0.5,

    translate=0.1,
    scale=0.3,
    shear=0.05,
    perspective=0.001,
    degrees=5.0,

    mixup=0.1,
    erasing=0.0,
    close_mosaic=2,
    dropout=0.1,
)

In [4]:
!pip install ultralytics --upgrade --quiet
import os
import shutil
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Step 1: Define paths
weights_path = "/kaggle/input/weight-v8/best.pt"  
predict_dir = "/kaggle/working/predict"
test_images_dir = "/kaggle/working/combined_split/test/images"
yaml_path = "/kaggle/working/rivet_detection.yaml"

# Step 2: Create predict folder
os.makedirs(predict_dir, exist_ok=True)

# Step 3: Copy weights to working directory if needed
working_weights_path = "/kaggle/input/weight-v8/best.pt"
if not os.path.exists(working_weights_path):
    shutil.copy(weights_path, working_weights_path)

# Step 4: Load the model
model = YOLO(working_weights_path)

# Step 5: Run prediction on test images
results = model.predict(
    source=test_images_dir,
    save=True,
    save_txt=True,
    conf=0.50,
    imgsz=896,
    project=predict_dir,
    name="predict_results",
    exist_ok=True
)

# Step 6: Run validation on test set to get confusion matrix
metrics = model.val(
    data=yaml_path,
    split='test',
    save_json=True,
    plots=True,
    project=predict_dir,
    name="val_results",
    exist_ok=True
)

print("✅ Inference complete.")


image 1/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0005_jpg.rf.2dd9fc700d1f491869c790bc6592912e_aug_1.jpg: 896x896 1 Absence_Tail, 6 Head_Rivets, 5 Tail_Rivets, 26.4ms
image 2/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0005_jpg.rf.2dd9fc700d1f491869c790bc6592912e_aug_3.jpg: 896x896 1 Absence_Tail, 6 Head_Rivets, 5 Tail_Rivets, 26.5ms
image 3/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0007_jpg.rf.36efe4178203e6b12bc53223001f5c4e.jpg: 896x896 1 Absence_Tail, 6 Head_Rivets, 5 Tail_Rivets, 26.4ms
image 4/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0007_jpg.rf.36efe4178203e6b12bc53223001f5c4e_aug_2.jpg: 896x896 1 Absence_Tail, 6 Head_Rivets, 5 Tail_Rivets, 26.5ms
image 5/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0009_jpg.rf.bc224a81eb1b115382dd6b58dc81aad5.jpg: 896x896 6 Head_Rivets, 6 Tail_Rivets, 20.2ms
image 6/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0009_jpg.rf.bc224a81eb1b1153

100%|██████████| 755k/755k [00:00<00:00, 16.4MB/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3216.8±378.1 MB/s, size: 494.0 KB)



val: Scanning /kaggle/working/combined_split/test/labels... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<00:00, 589.57it/s]

val: New cache created: /kaggle/working/combined_split/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.47s/it]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         50        600      0.952      0.995      0.991      0.684
          Absence_Head          7          7      0.873      0.984      0.978      0.651
          Absence_Tail         20         20      0.964          1      0.995      0.753
            Head_Rivet         50        292      0.992          1      0.995      0.692
            Tail_Rivet         50        281      0.977      0.996      0.995       0.64
Speed: 12.0ms preprocess, 25.7ms inference, 0.0ms loss, 25.7ms postprocess per image
Saving /kaggle/working/predict/val_results/predictions.json...
Results saved to /kaggle/working/predict/val_results
✅ Inference complete.


In [5]:
import shutil

# Zip the predict_tta folder
shutil.make_archive("/kaggle/working/predict/predict_results", 'zip', "/kaggle/working/predict")
print("✅ Folder zipped as /kaggle/working/predict.zip")

✅ Folder zipped as /kaggle/working/predict.zip
